In [59]:
with open('sample_text.txt','r') as f:
    lines=f.readlines()

In [32]:
import pdfplumber
text=[]
with pdfplumber.open('joker_new_final.pdf') as f :
    pages=f.pages
    for page in range(2,104):
        text.append(pages[page].extract_text(x_tolerance=3, y_tolerance=3, layout=True, x_density=7.25, y_density=13))
        
    f.close()

In [17]:
# with open('text.txt','w+' ) as t:
#     for i in text:
#         t.writelines(i)
with open('text.txt','r') as f:
    text=f.readlines()

In [13]:
import pandas as pd
import re

In [2]:
def get_characters(page):
    characters=[]
    for character in [e.strip() for e in page if e.isupper()]:
        if character not in TRANSITIONS and character.split()[0] not in INT_EXT:
            # characters.add(character)
            characters.append(character)
    return (characters )       
    # return (characters )       

In [ ]:
get_characters(text)

In [22]:
re.sub(r"[\(\)].+$",'','DET. BURKE (OS)').strip()

'DET. BURKE'

In [43]:
caarr=pd.DataFrame(get_characters(text),columns=['characters'])['characters'].apply(lambda x: re.sub(r"[\(\)].+$",'',x).strip())


In [ ]:
caarr.value_counts().head(50)

In [4]:
def get_scene(page):
    scenes=[]
    for i in [e.strip() for e in page if e.isupper() and e not in TRANSITIONS]:
        for item in i.split():
            if item in INT_EXT:
                scene=' '.join([x for x in i.split()])
                scenes.append(scene)
                
    return scenes 

In [ ]:
[s for s in [e.strip() for e in sample_text] if s not in TRANSITIONS and s.split()[0] not in INT_EXT and not s.isupper()]

In [23]:
[s for s in get_scene(text)]
    # print(text[get_scene(text).index(s)])


['INT. DEPT. OF HEALTH, OFFICE - MORNING',
 'EXT. GOTHAM SQUARE, MIDTOWN - AFTERNOON - DAYS LATER',
 'EXT. CORNER, ALLEY - GOTHAM SQUARE - CONTINUOUS',
 'INT. CITY BUS (MOVING) - HEADING UPTOWN - LATE AFTERNOON',
 'EXT. THE BRONX, STREET - SUNSET',
 'EXT. SIDE ALLEY, TENEMENTS - EARLY DUSK',
 'EXT. STEEP STAIRWAY, TENEMENTS - DUSK',
 'INT. APARTMENT BUILDING, LOBBY - DUSK',
 'INT. ELEVATOR, APARTMENT BUILDING - CONTINUOUS',
 "INT. MOM'S APARTMENT, FRONT DOOR - EVENING"]

In [ ]:
def get_action(page):
    act=[]
    action=[]
    
    for i in [e.split() for e in page]:
        if ' '.join(i) not in TRANSITIONS:
            action.append(' '.join(i))
            
    # for a in range(len(get_scene(page))-1):
    #     start_ind=action.index(get_scene(page)[a])+2
    #     end_ind=action.index(get_scene(page)[a+1])-2
    #     act.append([d for d in action[start_ind:end_ind] if len(d)>35])
    return action
    # return [' '.join(d) for d in [a for a in act]]

action=get_action(text)  
action

In [63]:
def get_dialog(page):
    act=[]
    dialog=[]
    
    for i in [e.split() for e in page if not e.isspace()]:
        if ' '.join(i) not in TRANSITIONS:
            dialog.append(' '.join(i))
            
    for a in range(len(get_characters(page))+1):
        start_ind=dialog.index(get_dialog(page)[a])+2
        end_ind=dialog.index(get_scene(page)[a+1])-2
        act.append([d for d in dialog[start_ind:end_ind] if len(d)<35])
        
    return [' '.join(d) for d in [a for a in act]]

get_dialog(text)  

## dialog extraction

In [40]:
'''
for each scene 
-strip the white space and only include lines with not more than 40 letters
-find indecies of lines of where each charachter appear
-use those indecies to extract the data inbetween characters

- need to make more find a way to accomodate
'''
def dialogs(scene,characte_list):

  res=[l.strip() for l in scene.splitlines() if l!='' and len(l.strip())<40 ]# optional
  inx=[i for i,l in enumerate(res) if l in characte_list]
  
  para=[]
  
  for i in range(len(inx)):
    if inx[i]==inx[-1]:
      para.append(res[inx[i]:])
    else:
      para.append(res[inx[i]:inx[i+1]])
  return para

In [ ]:
dialogs(scene)

In [45]:
class Screenplay():
    
    INT_EXT = ['INT.', 'EXT.', 'INT/EXT.', 'EXT/INT.']
    TOD = ['AFTERNOON', 'CONTINUOUS', 'DAWN', 'DAY', 'DUSK',
           'EVENING', 'LATER', 'MORNING', 'NIGHT', 'NOON', 'SAME']
    TRANSITIONS = ['BACK TO:', 'CUT TO:', 'DISSOLVE TO:', 'FADE IN:', 'CUT BACK TO:'
                   'FADE OUT:', 'FADE TO:', 'JUMP CUT TO:', 'MATCH CUT TO:', 'TITLE:',
                   'SMASH CUT TO:', 'TIME CUT:', 'WIPE TO:', 'CROSSFADE:', 'OVER BLACK:', 'CUT TO BLACK',
                   'FADE TO:BLACK', 'FLASH CUT:', 'LAP DISSOLVE:', 'MATCH DISSOLVE TO:', 'SUPER:', 'IRIS OUT:']
    SHOTS = ['AERIAL SHOT', 'ANGLE ON',  'CLOSE ON', 'CLOSER ANGLE', 'CONTINUOUS',
             'CONTRAZOOM', 'CRAWL',  'ESTABLISHING SHOT',
             'EXTREMELY LONG SHOT (XLS):', 'FAVOR ON','HEAR LAUGHTER.',
             'FLASHBACK:', 'FREEZE FRAME:', 'INSERT','DAYS LATER'
             'INTO FRAME:', 'INTO VIEW:', 'MOS', 'O.C. / O.S.',
             'POV', 'PULL BACK:', 'PULL FOCUS:', 'PUSH IN:',
             'REVERSE ANGLE',  'SPLIT SCREEN SHOT:', 'STOCK SHOT:',
             'TIGHT ON', 'TIME CUT', 'V.O.', 'ZOOM:']

    def __init__(self, text: list) -> list:
        # self.text = text
        self.text = [l.strip() for l in text if not l.isspace()]

    def get_scene_index(self):
        return [i for i,l in enumerate(self.text) if l in self.locations()]
    
    def locations(self):
        scenes = []
        for i in [e for e in self.text if e.isupper() and e not in self.TRANSITIONS]:
            for item in i.split():
                if item in self.INT_EXT:
                    scene = ' '.join([x for x in i.split()])
                    scenes.append(scene)
        return scenes
    
    def get_scenes(self):
        scenes=[]
        for i in range(len(self.get_scene_index())):
            if self.get_scene_index()[i]==self.get_scene_index()[-1]:
                scenes.append(self.text[self.get_scene_index()[i]:])
            else:
                scenes.append(self.text[self.get_scene_index()[i]:self.get_scene_index()[i+1]])
        return scenes    
    
    def get_characters(self):
        characters=[]
        for scene in self.get_scenes():
            characters.append([c for c in scene if len(c)<16 and 
                        c.isupper() and 
                        len(c)<15 and 
                        c not in self.TRANSITIONS and 
                        c.split()[0] not in self.INT_EXT])
        return characters                 

        
    def load(self):
        self.scenes_head={}
        for i,s in enumerate(self.locations()[:-1],start=1):
            self.scenes_head.update({
                i:{
                'int_ext':s.split()[0],
                'location':' '.join(s.split()[1:-2]),
                'tod':s.split()[-1],
                # 'action': self.get_action()[i-1]
                }})
   
        return self.scenes_head
    

In [46]:
with open('text.txt', 'r') as f:
    text = f.readlines()

sc = Screenplay(text[:300])
# sc.get_scenes()
# sc.get_scene_index()
# sc.load()
# sc.get_scenes()
sc.get_characters()

# scene_heads=sc.get_scenes()
# scene_heads

[['ARTHUR',
  'SOCIAL WORKER',
  'ARTHUR',
  'SOCIAL WORKER',
  'ARTHUR',
  'SOCIAL WORKER',
  'ARTHUR',
  'SOCIAL WORKER',
  'ARTHUR',
  'SOCIAL WORKER',
  'ARTHUR',
  'ARTHUR',
  'SOCIAL WORKER',
  'ARTHUR',
  'SOCIAL WORKER',
  'ARTHUR',
  'SOCIAL WORKER',
  'SOCIAL WORKER',
  'ARTHUR',
  'SOCIAL WORKER',
  'ARTHUR',
  'SOCIAL WORKER',
  'ARTHUR',
  'SOCIAL WORKER',
  'ARTHUR',
  'ARTHUR',
  'SOCIAL WORKER',
  'ARTHUR'],
 [],
 ['JOKER'],
 []]

In [ ]:
scenes={}
for i,s in enumerate(sc.get_scenes(),start=1):
    
    scenes.update({
        i:{
        f'int_ext':[l for l in s.split()][0],
        'location':' '.join([l for l in s.split()][1:-2]),
        'tod':[l for l in s.split()][-1]}})

# if contiuos grab before
scenes